In [9]:
# 예시로 들 고객번호를 보기 위한 셀임.
import pandas as pd
cluster_df = pd.read_csv('./dataset/cluster_result.csv')
df = pd.read_pickle('./dataset/PP_CUS_QUAR_NORMAL.pkl')
import_df = pd.concat([df['고객번호'], cluster_df['cluster']], axis=1)
import_df = import_df.fillna(9)
import_df.cluster = import_df.cluster.astype('int')
# import_df.to_csv('./dataset/import_customer.csv', index=False)
import_df

,고객번호,cluster
0,00001,0
1,00002,9
2,00003,1
3,00004,9
4,00006,2
...,...,...
18861,19333,9
18862,19334,1
18863,19335,2
18864,19336,2


In [3]:
# 입력받은 중분류명 리스트 중에서 구매지수가 높은 것 n개 뽑고, 그것들로 추천리스트를 적용하게 된다.

def get_prodcl_list(target_id = None,prodcl_list=None,n = 3,date = None):
    import cx_Oracle
    import pandas as pd
    import numpy as np
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    
    cursor.execute(f"SELECT 구분2, COUNT(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 중분류별구매건수, SUM(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 중분류별구매총액 FROM PURPROD,PRODCL_RENEW WHERE PURPROD.소분류코드 = PRODCL_RENEW.소분류코드 AND 고객번호 = {target_id} AND 구분2 IN{tuple(prodcl_list)} GROUP BY 구분2 ORDER BY 구분2")
    x = cursor.fetchall()
    col = ['중분류명','중분류별구매건수','중분류별구매총액']
    df = pd.DataFrame(x,columns=col)
    df.중분류별구매건수.replace(0,np.nan,inplace=True)

    df['평균단가'] = df['중분류별구매총액'] / df['중분류별구매건수']
    meanprice = df['평균단가'].mean()
    df['단가가중치'] = df['평균단가'].apply(lambda x: x/meanprice)
    df['중분류별구매건수'] = df['중분류별구매건수'] * df['단가가중치']
    df.drop(['평균단가','단가가중치'],axis=1,inplace=True)
    
    df['총구매건수'] = df['중분류별구매건수'].sum()
    df.총구매건수 = df.총구매건수.replace(0,np.nan)
    
    # 최대값은 10으로 준다.
    df['중분류별구매비율'] = df.중분류별구매건수 * 10 / df.총구매건수
    df = df.drop(['중분류별구매건수','중분류별구매총액','총구매건수'],axis=1)
    df = df.sort_values(by='중분류별구매비율',ascending=False)
    high_list = df.중분류명.tolist()
    return high_list[:n]

def get_scode(prodcl_list=None):
    import cx_Oracle
    import pandas as pd
    
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    cursor.execute(f"SELECT 구분2,소분류코드,소분류명 FROM PRODCL_RENEW WHERE 구분2 IN {tuple(prodcl_list)} ORDER BY 구분2, 소분류코드")
    x = cursor.fetchall()
    col = ['중분류명','iid','소분류명']
    df = pd.DataFrame(x,columns=col)
    return df

def get_data(target_id = None,prodcl_list=None,date = 20151001):
    import cx_Oracle
    import pandas as pd
    import numpy as np

    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    cursor.execute(f"CREATE VIEW TEMP1 AS SELECT 구분2, 소분류코드 FROM PRODCL_RENEW WHERE 구분2 IN {tuple(prodcl_list)}")
    cursor.execute(f"CREATE VIEW TEMP2 AS SELECT 고객번호, PURPROD.소분류코드, COUNT(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 상품별구매건수, SUM(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 상품별구매총액 FROM PURPROD, PRODCL_RENEW WHERE PURPROD.소분류코드 = PRODCL_RENEW.소분류코드 AND 구분2 IN {tuple(prodcl_list)} AND 고객번호 = {target_id} GROUP BY PURPROD.고객번호, PURPROD.소분류코드 ORDER BY PURPROD.고객번호, PURPROD.소분류코드")
    cursor.execute(f"SELECT NVL(고객번호,{target_id}) 고객번호, TEMP1.소분류코드, 상품별구매건수, 상품별구매총액 FROM TEMP2 RIGHT OUTER JOIN TEMP1 ON TEMP2.소분류코드 = TEMP1.소분류코드 ORDER BY 고객번호, TEMP2.소분류코드")
    t = cursor.fetchall()
    col = ['고객번호','소분류코드','상품별구매건수','상품별구매총액']
    df1 = pd.DataFrame(t,columns=col)
    df1['상품별구매건수'] = df1['상품별구매건수'].replace(0,np.nan)

    cursor.execute(f"SELECT 고객번호, PURPROD.소분류코드, COUNT(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 상품별구매건수, SUM(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 상품별구매총액 FROM PURPROD,PRODCL_RENEW WHERE PURPROD.소분류코드 = PRODCL_RENEW.소분류코드 AND 구분2 IN {tuple(prodcl_list)} AND 고객번호 <> {target_id} GROUP BY PURPROD.고객번호, PURPROD.소분류코드 ORDER BY PURPROD.고객번호, PURPROD.소분류코드")
    x = cursor.fetchall()
    col = ['고객번호','소분류코드','상품별구매건수','상품별구매총액']
    df = pd.DataFrame(x,columns=col)
    df = df.dropna()

    cursor.execute(f"DROP VIEW TEMP1")
    cursor.execute(f"DROP VIEW TEMP2")

    # 구매지수로 정의한 것을 만들기 위한 과정. 단가 > 단가가중치 > 가중치가 적용된 건수
    df = pd.concat([df1,df],axis=0).reset_index().drop('index',axis=1)
    df['상품별단가'] = df['상품별구매총액'] / df['상품별구매건수']
    meanprice = df['상품별단가'].mean()
    df['단가가중치'] = df['상품별단가'].apply(lambda x: x/meanprice)
    df['상품별구매건수'] = df['상품별구매건수'] * df['단가가중치']
    df.drop(['상품별단가','단가가중치'],axis=1,inplace=True)
    
    tdf = pd.DataFrame(df.groupby('고객번호')['상품별구매건수'].sum())
    tdf = tdf.reset_index()
    mdf = pd.merge(df,tdf,on='고객번호')
    mdf.columns =['고객번호','소분류코드','상품별구매건수','상품별구매총액','총구매건수']
    mdf.총구매건수 = mdf.총구매건수.replace(0,np.nan)
    
    # 최대값은 10으로 준다.
    mdf['상품별구매비율'] = mdf.상품별구매건수 * 10 / mdf.총구매건수
    mdf = mdf.drop(['상품별구매건수','상품별구매총액','총구매건수'],axis=1)
    return mdf

def get_between_day(date):
    import datetime
    from datetime import timedelta
    date = datetime.datetime.strptime(date, '%Y%m%d')
    year = date.year - 1
    month = date.month
    next_month = month + 1
    last_month_day = (datetime.date(year, month, 1) - timedelta(1)).strftime('%Y%m%d')
    if next_month == 13:
        next_month = 13
        year = year + 1
    next_month_day = (datetime.date(year, next_month, 1)).strftime('%Y%m%d')
    return last_month_day, next_month_day

# 작년 해당 월에 인기 많은 상품을 추출하기 위하여 추천하려는 달의 데이터를 다시 뽑아온다.
def past_data(prodcl_list=None,date1 = 20140930, date2 = 20141101):
    import cx_Oracle
    import pandas as pd
    
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    cursor.execute(f"SELECT 구분2, PURPROD.소분류코드, COUNT(CASE WHEN 구매일자 between {date1} and {date2} THEN 구매금액 END) 상품별구매건수, SUM(CASE WHEN 구매일자 between {date1} and {date2} THEN 구매금액 END) 상품별구매총액 FROM PURPROD,PRODCL_RENEW WHERE PURPROD.소분류코드 = PRODCL_RENEW.소분류코드 AND 구분2 IN{tuple(prodcl_list)} GROUP BY 구분2, PURPROD.소분류코드 ORDER BY 구분2, PURPROD.소분류코드")
    x = cursor.fetchall()
    col = ['중분류명','소분류코드','상품별구매건수','상품별구매총액']
    df = pd.DataFrame(x,columns=col)
    df = df.dropna()

    df['평균단가'] = df['상품별구매총액'] / df['상품별구매건수'] # 단가 계산
    meanprice = df['평균단가'].mean()
    df['단가가중치'] = df['평균단가'].apply(lambda x: x/meanprice) # 단가를 단가 평균으로 나눠주면 가중치가 나온다
    df['상품별구매건수'] = df['상품별구매건수'] * df['단가가중치'] # 건수에 가중치를 곱해준 것으로 새로운 건수를 만든다
    df.drop(['평균단가','단가가중치'],axis=1,inplace=True)

    tdf = pd.DataFrame(df.groupby('중분류명')['상품별구매건수'].sum()) # 새로운건수의 총합을 중분류별로 구함
    tdf = tdf.reset_index()
    mdf = pd.merge(df,tdf,on='중분류명')
    mdf.columns =['중분류명','소분류코드','상품별구매건수','상품별구매총액','총구매건수']
    mdf['상품별구매비율'] = mdf.상품별구매건수 * 10 / mdf.총구매건수
    
    mdf = mdf.drop(['상품별구매건수','상품별구매총액','총구매건수'],axis=1)
    return mdf

# surprise api 이용을 위해 데이터셋 구성하는 사용자 함수.
def convert_traintest_dataframe_forsurprise(uid = None, dataframe = None):
    from surprise import Reader
    from surprise import Dataset

    data = dataframe[dataframe.상품별구매비율.isnull() == False]
    target_data = dataframe[(dataframe.고객번호 == uid)]

    # 데이터셋을 train과 test로 나누기 위해 섞어준다.
    data = data.sample(frac=1,random_state=0)
    cutoff = int(.80 * len(data))
    train_data = data[:cutoff]
    test_data = data[cutoff:]

    # 최고점은 10점이므로 다음과 같이 구성하였다.
    reader = Reader(rating_scale=(0.001, 10))
    trainset = Dataset.load_from_df(train_data[['고객번호', '소분류코드', '상품별구매비율']], reader)
    testset = Dataset.load_from_df(test_data[['고객번호', '소분류코드', '상품별구매비율']], reader)
    targetset = Dataset.load_from_df(target_data[['고객번호', '소분류코드', '상품별구매비율']], reader)
    
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)

    targetset = targetset.construct_testset(targetset.raw_ratings)
    
    return trainset, testset, targetset

# Knn-baseline 모델 
def knn_item_based_recommendation(uid = None, trainset = None, testset = None):
    from surprise.prediction_algorithms import KNNBaseline
    from surprise import accuracy
    from surprise import dump
    import pandas as pd
    
    sim_options = {'name': 'cosine',
                'user_based': False  # content(item)-based similarity
                }
    bsl_options = {'method': 'sgd'}

    knnbaseline_algo_i = KNNBaseline(sim_options=sim_options,bsl_options=bsl_options)
    knnbaseline_algo_i.fit(trainset)
    knnbaseline_i_pred = knnbaseline_algo_i.test(testset)
    file_name = f'./dataset/KnnBaseline_model_i_{uid}'
    dump.dump(file_name, predictions=knnbaseline_i_pred, algo=knnbaseline_algo_i)
    
    knn_rmse = accuracy.rmse(knnbaseline_i_pred)
    
    return knn_rmse

# SVD
def svd_recommendation(uid = None, trainset = None, testset = None):
    from surprise import accuracy
    from surprise import dump
    from surprise import SVD
    import pandas as pd

    svd_algo = SVD()

    svd_algo.fit(trainset)
    svd_pred = svd_algo.test(testset)
    file_name = f'./dataset/svd_model_{uid}'
    dump.dump(file_name, predictions = svd_pred, algo=svd_algo)

    svd_rmse = accuracy.rmse(svd_pred)
    return svd_rmse

def best_algorism_result(uid = None, targetset = None):
    import pandas as pd
    from surprise import dump

    if knn_rmse > svd_rmse:
        print("SVD의 RMSE가 Knn Baseline의 RMSE보다 낮아 SVD 기반 추천 시스템을 적용합니다.")
        _, loaded_algo = dump.load(f'./dataset/svd_model_{uid}')
    else:
        print("Knn Baseline의 RMSE가 SVD의 RMSE보다 낮아 KNN 기반 추천 시스템을 적용합니다.")
        _, loaded_algo = dump.load(f'./dataset/KnnBaseline_model_i_{uid}')

    target_pred = loaded_algo.test(targetset)
    result = pd.DataFrame(target_pred)
    result = result.drop('details',axis=1).sort_values('est',ascending=False)
    result = result.merge(prodcl_scode,on='iid')
    return result

def main(target_id=None, uid=None, prodcl_list_ori = None, date=None):
    global prodcl_list ; prodcl_list = get_prodcl_list(target_id=target_id, prodcl_list=prodcl_list_ori, date = date)
    global prodcl_scode ; prodcl_scode = get_scode(prodcl_list)
    data = get_data(target_id=target_id,prodcl_list=prodcl_list)
    trainset, testset, targetset = convert_traintest_dataframe_forsurprise(uid,data)
    global knn_rmse ; knn_rmse = knn_item_based_recommendation(uid,trainset,testset)
    global svd_rmse ; svd_rmse = svd_recommendation(uid,trainset,testset)
    result = best_algorism_result(uid, targetset)
    return result

def main_contain_bestseller(target_id=None, uid=None, prodcl_list_ori=None, date=None):
    result = main(target_id=target_id, uid=uid, prodcl_list_ori=prodcl_list_ori, date=date)
    date1, date2 = get_between_day(str(date))
    past = past_data(prodcl_list,date1,date2)
    past.columns = ['중분류명','iid','r_ui']
    past = past.merge(prodcl_scode.drop('중분류명',axis=1),on='iid')

    result_new = result.copy()
    for i in prodcl_list:
        iid_li = past.iid[past.중분류명 == i][:10].tolist()
        for j in iid_li:
            result_new.loc[result_new.iid == j,'est'] = result_new.loc[result_new.iid == j,'est'] * 1.5
    result_new = result_new.sort_values('est',ascending=False)
    return result_new


In [5]:
# 타겟 정의
def change_target_id(target_id):
    target_id = target_id.replace("'","")
    uid = target_id.replace('"','')
    return uid

target_id = "'00022'"
uid = change_target_id(target_id)
prodcl_list=['패션/의류 고가','생활 고가','가구 저가']
date = 20151001

In [7]:
# 이 예시는 47초 걸렸음.
result = main(target_id=target_id, uid=uid, prodcl_list_ori=prodcl_list, date=date)
display(result)

Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.2003
RMSE: 1.9871
SVD의 RMSE가 Knn Baseline의 RMSE보다 낮아 SVD 기반 추천 시스템을 적용합니다.


,uid,iid,r_ui,est,중분류명,소분류명
0,00022,B410102,NaN,8.030208,생활 고가,압력밥솥
1,00022,A060202,8.552832,7.778189,패션/의류 고가,L/C골프의류
2,00022,B860301,NaN,6.314499,생활 고가,프리미엄다용도매트
3,00022,A020601,NaN,6.254804,패션/의류 고가,네셔널
4,00022,B200203,NaN,5.092108,생활 고가,온라인 음식조리
...,...,...,...,...,...,...
109,00022,A040307,NaN,1.130024,패션/의류 고가,안토니모라토 상품군
110,00022,A090406,NaN,1.102616,생활 고가,식기단기행사
111,00022,A050111,NaN,1.080168,패션/의류 고가,니나리찌
112,00022,A090401,NaN,1.061913,생활 고가,크리스탈


In [8]:
# 이 예시는 1분 걸렸음
result_new = main_contain_bestseller(target_id=target_id, uid=uid, prodcl_list_ori=prodcl_list, date=date)
display(result_new)

Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.2003
RMSE: 1.9787
SVD의 RMSE가 Knn Baseline의 RMSE보다 낮아 SVD 기반 추천 시스템을 적용합니다.


,uid,iid,r_ui,est,중분류명,소분류명
2,00022,A020601,NaN,10.428669,패션/의류 고가,네셔널
0,00022,A060202,8.552832,7.762377,패션/의류 고가,L/C골프의류
1,00022,B410102,NaN,7.458132,생활 고가,압력밥솥
5,00022,A040222,NaN,7.185191,패션/의류 고가,영 캐릭터
7,00022,A040212,NaN,6.380233,패션/의류 고가,수입브릿지
...,...,...,...,...,...,...
107,00022,A050102,NaN,1.191324,패션/의류 고가,직수입정장
108,00022,A040307,NaN,1.152865,패션/의류 고가,안토니모라토 상품군
110,00022,A040232,NaN,1.068648,패션/의류 고가,프리미엄진
113,00022,A090306,NaN,1.029838,생활 고가,매트
